In [1]:
import pandas as pd
import numpy as np

In [ ]:
# Load the taxi zone geometry data
geometry_df = pd.read_csv('taxi_zone_geometry.csv')

# Display basic information about the dataset
print("Dataset Info:")
print(f"Shape: {geometry_df.shape}")
print(f"Columns: {list(geometry_df.columns)}")
print("\nFirst 5 rows:")
print(geometry_df.head())

In [3]:
# Find duplicated zone_ids
duplicated_zone_ids = geometry_df[geometry_df.duplicated(subset=['zone_id'], keep=False)]

if len(duplicated_zone_ids) > 0:
    print(f"Found {len(duplicated_zone_ids)} rows with duplicated zone_ids")
    print("\nDuplicated entries:")
    print(duplicated_zone_ids[['zone_id', 'zone_name', 'borough']].sort_values('zone_id'))
    
    # Group by zone_id to see which specific IDs are duplicated
    duplicate_groups = duplicated_zone_ids.groupby('zone_id').size().sort_index()
    print(f"\nZone IDs with duplicates (count per ID):")
    for zone_id, count in duplicate_groups.items():
        print(f"Zone ID {zone_id}: {count} occurrences")
        zone_details = geometry_df[geometry_df['zone_id'] == zone_id][['zone_id', 'zone_name', 'borough']]
        for _, row in zone_details.iterrows():
            print(f"  - {row['zone_name']}, {row['borough']}")
        print()
        
else:
    print("No duplicated zone_ids found!")

Found 5 rows with duplicated zone_ids

Duplicated entries:
     zone_id                                      zone_name    borough
57        56                                         Corona     Queens
58        56                                         Corona     Queens
192      103  Governor's Island/Ellis Island/Liberty Island  Manhattan
193      103  Governor's Island/Ellis Island/Liberty Island  Manhattan
194      103  Governor's Island/Ellis Island/Liberty Island  Manhattan

Zone IDs with duplicates (count per ID):
Zone ID 56: 2 occurrences
  - Corona, Queens
  - Corona, Queens

Zone ID 103: 3 occurrences
  - Governor's Island/Ellis Island/Liberty Island, Manhattan
  - Governor's Island/Ellis Island/Liberty Island, Manhattan
  - Governor's Island/Ellis Island/Liberty Island, Manhattan



In [4]:
print(f"Total unique zone_ids: {geometry_df['zone_id'].nunique()}")
print(f"Total rows: {len(geometry_df)}")
print(f"Missing zone_ids: {geometry_df['zone_id'].isna().sum()}")

# Show zone_id range
print(f"Zone ID range: {geometry_df['zone_id'].min()} to {geometry_df['zone_id'].max()}")

# Borough distribution
print(f"\nBorough distribution:")
print(geometry_df['borough'].value_counts().sort_index())

Total unique zone_ids: 260
Total rows: 263
Missing zone_ids: 0
Zone ID range: 1 to 263

Borough distribution:
borough
Bronx            43
Brooklyn         61
EWR               1
Manhattan        69
Queens           69
Staten Island    20
Name: count, dtype: int64


In [5]:
# Remove duplicated zone_id rows, keeping only the first occurrence
geometry_df_cleaned = geometry_df.drop_duplicates(subset=['zone_id'], keep='first')

print(f"Original dataset: {len(geometry_df)} rows")
print(f"After removing duplicates: {len(geometry_df_cleaned)} rows")
print(f"Removed {len(geometry_df) - len(geometry_df_cleaned)} duplicate rows")

# Verify no duplicates remain
remaining_duplicates = geometry_df_cleaned[geometry_df_cleaned.duplicated(subset=['zone_id'], keep=False)]
print(f"Remaining duplicates: {len(remaining_duplicates)}")

# Save the cleaned dataset
geometry_df_cleaned.to_csv('taxi_zone_preprocessed.csv', index=False)
print("\nCleaned dataset saved as 'taxi_zone_preprocessed.csv'")

Original dataset: 263 rows
After removing duplicates: 260 rows
Removed 3 duplicate rows
Remaining duplicates: 0

Cleaned dataset saved as 'taxi_zone_preprocessed.csv'


### yellow_taxi_data

In [12]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq

# Read specific parquet file using PyArrow directly
parquet_file = 'D:/tlc_yellow_taxi/2023/yellow_tripdata_2023-01.parquet'

# Read the parquet file
table = pq.read_table(parquet_file)
taxi_df = table.to_pandas()

print(f"Number of records: {len(taxi_df):,}")
print(f"Number of columns: {len(taxi_df.columns)}")
print(f"Shape: {taxi_df.shape}")

print(f"\nColumns: {list(taxi_df.columns)}")
print(f"\nFirst 5 rows:")
print(taxi_df.head())

# Additional info about the parquet file
print(f"\nParquet file info:")
print(f"File size: {table.nbytes / 1024**2:.2f} MB")
print(f"Schema:")
print(table.schema)

Number of records: 3,066,766
Number of columns: 19
Shape: (3066766, 19)

Columns: ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee']

First 5 rows:
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2023-01-01 00:32:10   2023-01-01 00:40:36              1.0   
1         2  2023-01-01 00:55:08   2023-01-01 01:01:27              1.0   
2         2  2023-01-01 00:25:04   2023-01-01 00:37:49              1.0   
3         1  2023-01-01 00:03:48   2023-01-01 00:13:25              0.0   
4         2  2023-01-01 00:10:29   2023-01-01 00:21:19              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           0.97         1.0                  

In [13]:
# Check for invalid location IDs in taxi data
print("="*60)
print("INVALID LOCATION ID ANALYSIS")
print("="*60)

# Define valid zone_id range
min_zone_id = 1
max_zone_id = 263
print(f"Valid zone_id range: {min_zone_id} to {max_zone_id}")

# Check for invalid PULocationID
invalid_pickup = taxi_df[
    (taxi_df['PULocationID'] < min_zone_id) | 
    (taxi_df['PULocationID'] > max_zone_id) |
    (taxi_df['PULocationID'].isna())
]

# Check for invalid DOLocationID  
invalid_dropoff = taxi_df[
    (taxi_df['DOLocationID'] < min_zone_id) | 
    (taxi_df['DOLocationID'] > max_zone_id) |
    (taxi_df['DOLocationID'].isna())
]

# Check for records with either invalid pickup OR dropoff
invalid_either = taxi_df[
    (taxi_df['PULocationID'] < min_zone_id) | 
    (taxi_df['PULocationID'] > max_zone_id) |
    (taxi_df['PULocationID'].isna()) |
    (taxi_df['DOLocationID'] < min_zone_id) | 
    (taxi_df['DOLocationID'] > max_zone_id) |
    (taxi_df['DOLocationID'].isna())
]

print(f"\nInvalid Records Summary:")
print(f"Records with invalid PULocationID: {len(invalid_pickup):,}")
print(f"Records with invalid DOLocationID: {len(invalid_dropoff):,}")
print(f"Records with either invalid location: {len(invalid_either):,}")
print(f"Percentage of invalid records: {len(invalid_either)/len(taxi_df)*100:.2f}%")

# Show details of invalid pickup locations
if len(invalid_pickup) > 0:
    print(f"\n--- Invalid PULocationID Details ---")
    pickup_counts = invalid_pickup['PULocationID'].value_counts().sort_index()
    print(f"Unique invalid pickup location IDs: {pickup_counts.index.tolist()}")
    print("Count by invalid pickup location ID:")
    for loc_id, count in pickup_counts.head(10).items():
        print(f"  PULocationID {loc_id}: {count:,} records")
    if len(pickup_counts) > 10:
        print(f"  ... and {len(pickup_counts) - 10} more")

# Show details of invalid dropoff locations
if len(invalid_dropoff) > 0:
    print(f"\n--- Invalid DOLocationID Details ---")
    dropoff_counts = invalid_dropoff['DOLocationID'].value_counts().sort_index()
    print(f"Unique invalid dropoff location IDs: {dropoff_counts.index.tolist()}")
    print("Count by invalid dropoff location ID:")
    for loc_id, count in dropoff_counts.head(10).items():
        print(f"  DOLocationID {loc_id}: {count:,} records")
    if len(dropoff_counts) > 10:
        print(f"  ... and {len(dropoff_counts) - 10} more")

# Show sample records with invalid locations
if len(invalid_either) > 0:
    print(f"\n--- Sample Invalid Records ---")
    sample_cols = ['tpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance', 'fare_amount']
    print(invalid_either[sample_cols].head(10))

# Check for null values specifically
null_pickup = taxi_df['PULocationID'].isna().sum()
null_dropoff = taxi_df['DOLocationID'].isna().sum()

print(f"\n--- Null/Missing Location IDs ---")
print(f"Records with null PULocationID: {null_pickup:,}")
print(f"Records with null DOLocationID: {null_dropoff:,}")

INVALID LOCATION ID ANALYSIS
Valid zone_id range: 1 to 263

Invalid Records Summary:
Records with invalid PULocationID: 41,763
Records with invalid DOLocationID: 33,549
Records with either invalid location: 58,248
Percentage of invalid records: 1.90%

--- Invalid PULocationID Details ---
Unique invalid pickup location IDs: [264, 265]
Count by invalid pickup location ID:
  PULocationID 264: 40,116 records
  PULocationID 265: 1,647 records

--- Invalid DOLocationID Details ---
Unique invalid dropoff location IDs: [264, 265]
Count by invalid dropoff location ID:
  DOLocationID 264: 22,591 records
  DOLocationID 265: 10,958 records

--- Sample Invalid Records ---
    tpep_pickup_datetime  PULocationID  DOLocationID  trip_distance  \
11   2023-01-01 00:43:37            79           264           7.30   
59   2023-01-01 00:22:39           132           265          16.02   
98   2023-01-01 00:10:50           264           186           1.41   
99   2023-01-01 00:27:35           264          

In [14]:
# Further Data Analysis
print("="*80)
print("COMPREHENSIVE TAXI DATA ANALYSIS")
print("="*80)

# 1. TEMPORAL ANALYSIS
print("\n1. TEMPORAL ANALYSIS")
print("-" * 40)

# Convert datetime columns
taxi_df['tpep_pickup_datetime'] = pd.to_datetime(taxi_df['tpep_pickup_datetime'])
taxi_df['tpep_dropoff_datetime'] = pd.to_datetime(taxi_df['tpep_dropoff_datetime'])

# Calculate trip duration
taxi_df['trip_duration_minutes'] = (taxi_df['tpep_dropoff_datetime'] - taxi_df['tpep_pickup_datetime']).dt.total_seconds() / 60

print(f"Date range: {taxi_df['tpep_pickup_datetime'].min()} to {taxi_df['tpep_pickup_datetime'].max()}")

# Daily trip counts
daily_trips = taxi_df['tpep_pickup_datetime'].dt.date.value_counts().sort_index()
print(f"Average trips per day: {daily_trips.mean():.0f}")
print(f"Min daily trips: {daily_trips.min()}")
print(f"Max daily trips: {daily_trips.max()}")

# Hourly distribution
hourly_trips = taxi_df['tpep_pickup_datetime'].dt.hour.value_counts().sort_index()
print(f"\nPeak hour: {hourly_trips.idxmax()}:00 with {hourly_trips.max():,} trips")
print(f"Lowest hour: {hourly_trips.idxmin()}:00 with {hourly_trips.min():,} trips")

# 2. TRIP CHARACTERISTICS
print("\n2. TRIP CHARACTERISTICS")
print("-" * 40)

# Trip distance analysis
print("Trip Distance Statistics:")
print(f"  Mean: {taxi_df['trip_distance'].mean():.2f} miles")
print(f"  Median: {taxi_df['trip_distance'].median():.2f} miles")
print(f"  Min: {taxi_df['trip_distance'].min():.2f} miles")
print(f"  Max: {taxi_df['trip_distance'].max():.2f} miles")
print(f"  Zero distance trips: {(taxi_df['trip_distance'] == 0).sum():,}")

# Trip duration analysis
print("\nTrip Duration Statistics:")
print(f"  Mean: {taxi_df['trip_duration_minutes'].mean():.2f} minutes")
print(f"  Median: {taxi_df['trip_duration_minutes'].median():.2f} minutes")
print(f"  Min: {taxi_df['trip_duration_minutes'].min():.2f} minutes")
print(f"  Max: {taxi_df['trip_duration_minutes'].max():.2f} minutes")

# Identify unusual trips
short_trips = taxi_df[taxi_df['trip_duration_minutes'] <= 1]
long_trips = taxi_df[taxi_df['trip_duration_minutes'] >= 180]  # 3+ hours
print(f"  Very short trips (≤1 min): {len(short_trips):,}")
print(f"  Very long trips (≥3 hours): {len(long_trips):,}")

# 3. FARE ANALYSIS
print("\n3. FARE ANALYSIS")
print("-" * 40)

fare_cols = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount']
for col in fare_cols:
    if col in taxi_df.columns:
        print(f"{col}:")
        print(f"  Mean: ${taxi_df[col].mean():.2f}")
        print(f"  Median: ${taxi_df[col].median():.2f}")
        print(f"  Negative values: {(taxi_df[col] < 0).sum():,}")
        print(f"  Zero values: {(taxi_df[col] == 0).sum():,}")

# 4. PAYMENT ANALYSIS
print("\n4. PAYMENT ANALYSIS")
print("-" * 40)

if 'payment_type' in taxi_df.columns:
    payment_counts = taxi_df['payment_type'].value_counts().sort_index()
    print("Payment Type Distribution:")
    for payment_type, count in payment_counts.items():
        percentage = (count / len(taxi_df)) * 100
        print(f"  Type {payment_type}: {count:,} ({percentage:.1f}%)")

# 5. PASSENGER ANALYSIS
print("\n5. PASSENGER ANALYSIS")
print("-" * 40)

if 'passenger_count' in taxi_df.columns:
    passenger_counts = taxi_df['passenger_count'].value_counts().sort_index()
    print("Passenger Count Distribution:")
    for passengers, count in passenger_counts.items():
        percentage = (count / len(taxi_df)) * 100
        print(f"  {passengers} passenger(s): {count:,} ({percentage:.1f}%)")
    
    # Check for unusual passenger counts
    zero_passengers = (taxi_df['passenger_count'] == 0).sum()
    high_passengers = (taxi_df['passenger_count'] > 6).sum()
    print(f"\nUnusual passenger counts:")
    print(f"  Zero passengers: {zero_passengers:,}")
    print(f"  More than 6 passengers: {high_passengers:,}")

# 6. LOCATION ANALYSIS
print("\n6. LOCATION ANALYSIS")
print("-" * 40)

# Most popular pickup locations
top_pickup = taxi_df['PULocationID'].value_counts().head(10)
print("Top 10 Pickup Locations:")
for i, (location_id, count) in enumerate(top_pickup.items(), 1):
    percentage = (count / len(taxi_df)) * 100
    print(f"  {i}. Zone {location_id}: {count:,} trips ({percentage:.1f}%)")

# Most popular dropoff locations  
top_dropoff = taxi_df['DOLocationID'].value_counts().head(10)
print("\nTop 10 Dropoff Locations:")
for i, (location_id, count) in enumerate(top_dropoff.items(), 1):
    percentage = (count / len(taxi_df)) * 100
    print(f"  {i}. Zone {location_id}: {count:,} trips ({percentage:.1f}%)")

# 7. DATA QUALITY ISSUES
print("\n7. DATA QUALITY ISSUES")
print("-" * 40)

# Check for missing values
print("Missing Values by Column:")
missing_data = taxi_df.isnull().sum()
for col, missing_count in missing_data.items():
    if missing_count > 0:
        percentage = (missing_count / len(taxi_df)) * 100
        print(f"  {col}: {missing_count:,} ({percentage:.2f}%)")

# Check for logical inconsistencies
pickup_after_dropoff = taxi_df[taxi_df['tpep_pickup_datetime'] > taxi_df['tpep_dropoff_datetime']]
print(f"\nLogical Issues:")
print(f"  Pickup after dropoff: {len(pickup_after_dropoff):,}")
print(f"  Negative trip distances: {(taxi_df['trip_distance'] < 0).sum():,}")
print(f"  Negative durations: {(taxi_df['trip_duration_minutes'] < 0).sum():,}")

print("\n" + "="*80)
print("ANALYSIS COMPLETE")
print("="*80)

COMPREHENSIVE TAXI DATA ANALYSIS

1. TEMPORAL ANALYSIS
----------------------------------------
Date range: 2008-12-31 23:01:42 to 2023-02-01 00:56:53
Average trips per day: 85188
Min daily trips: 2
Max daily trips: 114877

Peak hour: 18:00 with 215,889 trips
Lowest hour: 4:00 with 17,835 trips

2. TRIP CHARACTERISTICS
----------------------------------------
Trip Distance Statistics:
  Mean: 3.85 miles
  Median: 1.80 miles
  Min: 0.00 miles
  Max: 258928.15 miles
  Zero distance trips: 45,862

Trip Duration Statistics:
  Mean: 15.67 minutes
  Median: 11.52 minutes
  Min: -29.20 minutes
  Max: 10029.18 minutes
  Very short trips (≤1 min): 33,616
  Very long trips (≥3 hours): 3,048

3. FARE ANALYSIS
----------------------------------------
fare_amount:
  Mean: $18.37
  Median: $12.80
  Negative values: 25,049
  Zero values: 1,110
extra:
  Mean: $1.54
  Median: $1.00
  Negative values: 12,407
  Zero values: 1,240,718
mta_tax:
  Mean: $0.49
  Median: $0.50
  Negative values: 24,501
  Zero

In [17]:
# Analyze records with pickup/dropoff times not in January 2023
print("="*70)
print("TEMPORAL DATA QUALITY ANALYSIS - NON-JANUARY 2023 RECORDS")
print("="*70)

# Convert datetime columns if not already done
taxi_df['tpep_pickup_datetime'] = pd.to_datetime(taxi_df['tpep_pickup_datetime'])
taxi_df['tpep_dropoff_datetime'] = pd.to_datetime(taxi_df['tpep_dropoff_datetime'])

# Extract year and month from pickup and dropoff datetime
taxi_df['pickup_year'] = taxi_df['tpep_pickup_datetime'].dt.year
taxi_df['pickup_month'] = taxi_df['tpep_pickup_datetime'].dt.month
taxi_df['dropoff_year'] = taxi_df['tpep_dropoff_datetime'].dt.year
taxi_df['dropoff_month'] = taxi_df['tpep_dropoff_datetime'].dt.month

# Create year-month combinations for easier analysis
taxi_df['pickup_year_month'] = taxi_df['tpep_pickup_datetime'].dt.to_period('M')
taxi_df['dropoff_year_month'] = taxi_df['tpep_dropoff_datetime'].dt.to_period('M')

# Find records with pickup time not in January 2023
pickup_not_jan_2023 = taxi_df[
    (taxi_df['pickup_year'] != 2023) | 
    (taxi_df['pickup_month'] != 1)
]

# Find records with dropoff time not in January 2023
dropoff_not_jan_2023 = taxi_df[
    (taxi_df['dropoff_year'] != 2023) | 
    (taxi_df['dropoff_month'] != 1)
]

# Find records with either pickup OR dropoff not in January 2023
either_not_jan_2023 = taxi_df[
    (taxi_df['pickup_year'] != 2023) | 
    (taxi_df['pickup_month'] != 1) |
    (taxi_df['dropoff_year'] != 2023) | 
    (taxi_df['dropoff_month'] != 1)
]

print(f"Total records: {len(taxi_df):,}")
print(f"\nRecords with pickup time NOT in January 2023: {len(pickup_not_jan_2023):,}")
print(f"Records with dropoff time NOT in January 2023: {len(dropoff_not_jan_2023):,}")
print(f"Records with either pickup OR dropoff NOT in January 2023: {len(either_not_jan_2023):,}")
print(f"Percentage of records with temporal issues: {len(either_not_jan_2023)/len(taxi_df)*100:.2f}%")

# Analyze pickup year-month distribution for non-January 2023
if len(pickup_not_jan_2023) > 0:
    print(f"\n--- Pickup Year-Month Distribution (Non-January 2023) ---")
    pickup_period_counts = pickup_not_jan_2023['pickup_year_month'].value_counts().sort_index()
    print("Top 10 pickup periods:")
    for period, count in pickup_period_counts.head(10).items():
        print(f"  {period}: {count:,} records")
    
    if len(pickup_period_counts) > 10:
        print(f"  ... and {len(pickup_period_counts) - 10} more periods")
    
    # Show date range for non-January 2023 pickups
    print(f"\nPickup date range for non-January 2023 records:")
    print(f"  Earliest: {pickup_not_jan_2023['tpep_pickup_datetime'].min()}")
    print(f"  Latest: {pickup_not_jan_2023['tpep_pickup_datetime'].max()}")

# Analyze dropoff year-month distribution for non-January 2023
if len(dropoff_not_jan_2023) > 0:
    print(f"\n--- Dropoff Year-Month Distribution (Non-January 2023) ---")
    dropoff_period_counts = dropoff_not_jan_2023['dropoff_year_month'].value_counts().sort_index()
    print("Top 10 dropoff periods:")
    for period, count in dropoff_period_counts.head(10).items():
        print(f"  {period}: {count:,} records")
    
    if len(dropoff_period_counts) > 10:
        print(f"  ... and {len(dropoff_period_counts) - 10} more periods")
    
    # Show date range for non-January 2023 dropoffs
    print(f"\nDropoff date range for non-January 2023 records:")
    print(f"  Earliest: {dropoff_not_jan_2023['tpep_dropoff_datetime'].min()}")
    print(f"  Latest: {dropoff_not_jan_2023['tpep_dropoff_datetime'].max()}")

# Show sample records with temporal issues
if len(either_not_jan_2023) > 0:
    print(f"\n--- Sample Records with Temporal Issues ---")
    sample_cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'pickup_year_month', 'dropoff_year_month', 'trip_distance', 'fare_amount']
    print(either_not_jan_2023[sample_cols].head(10))

# Check for specific temporal anomalies
print(f"\n--- Specific Temporal Anomalies ---")

# Records from other months in 2023
other_months_2023_pickup = taxi_df[
    (taxi_df['pickup_year'] == 2023) & 
    (taxi_df['pickup_month'] != 1)
]

other_months_2023_dropoff = taxi_df[
    (taxi_df['dropoff_year'] == 2023) & 
    (taxi_df['dropoff_month'] != 1)
]

# Records from other years
other_years_pickup = taxi_df[taxi_df['pickup_year'] != 2023]
other_years_dropoff = taxi_df[taxi_df['dropoff_year'] != 2023]

print(f"Records with pickup in other months of 2023: {len(other_months_2023_pickup):,}")
print(f"Records with dropoff in other months of 2023: {len(other_months_2023_dropoff):,}")
print(f"Records with pickup in other years: {len(other_years_pickup):,}")
print(f"Records with dropoff in other years: {len(other_years_dropoff):,}")

# Check for records that span across months
cross_month_trips = taxi_df[
    (taxi_df['pickup_year_month'] != taxi_df['dropoff_year_month'])
]

print(f"\nTrips that span across different months: {len(cross_month_trips):,}")

if len(cross_month_trips) > 0:
    print("Sample cross-month trips:")
    cross_sample = cross_month_trips[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'pickup_year_month', 'dropoff_year_month']].head(5)
    print(cross_sample)

# Summary of January 2023 data
records_jan_2023 = taxi_df[
    (taxi_df['pickup_year'] == 2023) & 
    (taxi_df['pickup_month'] == 1) &
    (taxi_df['dropoff_year'] == 2023) & 
    (taxi_df['dropoff_month'] == 1)
]

print(f"\n--- January 2023 Data Summary ---")
print(f"Records with both pickup and dropoff in January 2023: {len(records_jan_2023):,}")
print(f"Percentage of valid January 2023 records: {len(records_jan_2023)/len(taxi_df)*100:.2f}%")

# Additional January 2023 analysis
if len(records_jan_2023) > 0:
    print(f"\nJanuary 2023 date range:")
    print(f"  Earliest pickup: {records_jan_2023['tpep_pickup_datetime'].min()}")
    print(f"  Latest pickup: {records_jan_2023['tpep_pickup_datetime'].max()}")
    print(f"  Earliest dropoff: {records_jan_2023['tpep_dropoff_datetime'].min()}")
    print(f"  Latest dropoff: {records_jan_2023['tpep_dropoff_datetime'].max()}")

print("\n" + "="*70)
print("JANUARY 2023 TEMPORAL ANALYSIS COMPLETE")
print("="*70)

TEMPORAL DATA QUALITY ANALYSIS - NON-JANUARY 2023 RECORDS
Total records: 3,066,766

Records with pickup time NOT in January 2023: 48
Records with dropoff time NOT in January 2023: 642
Records with either pickup OR dropoff NOT in January 2023: 655
Percentage of records with temporal issues: 0.02%

--- Pickup Year-Month Distribution (Non-January 2023) ---
Top 10 pickup periods:
  2008-12: 2 records
  2022-10: 11 records
  2022-12: 25 records
  2023-02: 10 records

Pickup date range for non-January 2023 records:
  Earliest: 2008-12-31 23:01:42
  Latest: 2023-02-01 00:56:53

--- Dropoff Year-Month Distribution (Non-January 2023) ---
Top 10 dropoff periods:
  2009-01: 2 records
  2022-10: 11 records
  2022-12: 12 records
  2023-02: 617 records

Dropoff date range for non-January 2023 records:
  Earliest: 2009-01-01 14:29:11
  Latest: 2023-02-02 09:28:47

--- Sample Records with Temporal Issues ---
     tpep_pickup_datetime tpep_dropoff_datetime pickup_year_month  \
80    2022-12-31 23:51:30

In [20]:
# Data Cleaning Process
print("="*80)
print("TAXI DATA CLEANING PROCESS")
print("="*80)

# Start with original dataset
print(f"Original dataset size: {len(taxi_df):,} records")

# Keep track of cleaning steps
cleaning_steps = []

# 1. Remove records with pickup/dropoff times NOT in January 2023
print("\n1. REMOVING RECORDS NOT IN JANUARY 2023")
print("-" * 50)

# Ensure datetime columns are properly converted
taxi_df['tpep_pickup_datetime'] = pd.to_datetime(taxi_df['tpep_pickup_datetime'])
taxi_df['tpep_dropoff_datetime'] = pd.to_datetime(taxi_df['tpep_dropoff_datetime'])

# Extract year and month
taxi_df['pickup_year'] = taxi_df['tpep_pickup_datetime'].dt.year
taxi_df['pickup_month'] = taxi_df['tpep_pickup_datetime'].dt.month
taxi_df['dropoff_year'] = taxi_df['tpep_dropoff_datetime'].dt.year
taxi_df['dropoff_month'] = taxi_df['tpep_dropoff_datetime'].dt.month

# Create mask for January 2023 records (both pickup AND dropoff must be in Jan 2023)
jan_2023_mask = (
    (taxi_df['pickup_year'] == 2023) & 
    (taxi_df['pickup_month'] == 1) &
    (taxi_df['dropoff_year'] == 2023) & 
    (taxi_df['dropoff_month'] == 1)
)

non_jan_2023_count = len(taxi_df) - jan_2023_mask.sum()
print(f"Records NOT in January 2023: {non_jan_2023_count:,}")

taxi_df_clean = taxi_df[jan_2023_mask].copy()
cleaning_steps.append(f"Non-January 2023 records: {non_jan_2023_count:,}")
print(f"Remaining records: {len(taxi_df_clean):,}")

# 2. Remove records with invalid PULocationID or DOLocationID
print("\n2. REMOVING RECORDS WITH INVALID LOCATION IDs")
print("-" * 50)

# Define valid zone_id range
min_zone_id = 1
max_zone_id = 263

# Create mask for valid location IDs
valid_locations_mask = (
    (taxi_df_clean['PULocationID'] >= min_zone_id) & 
    (taxi_df_clean['PULocationID'] <= max_zone_id) &
    (taxi_df_clean['DOLocationID'] >= min_zone_id) & 
    (taxi_df_clean['DOLocationID'] <= max_zone_id) &
    (taxi_df_clean['PULocationID'].notna()) &
    (taxi_df_clean['DOLocationID'].notna())
)

invalid_locations_count = len(taxi_df_clean) - valid_locations_mask.sum()
print(f"Records with invalid location IDs: {invalid_locations_count:,}")

taxi_df_clean = taxi_df_clean[valid_locations_mask].copy()
cleaning_steps.append(f"Invalid location IDs: {invalid_locations_count:,}")
print(f"Remaining records: {len(taxi_df_clean):,}")

# 3. Remove zero distance trips
print("\n3. REMOVING ZERO DISTANCE TRIPS")
print("-" * 50)

zero_distance_mask = taxi_df_clean['trip_distance'] > 0
zero_distance_count = len(taxi_df_clean) - zero_distance_mask.sum()
print(f"Records with zero distance: {zero_distance_count:,}")

taxi_df_clean = taxi_df_clean[zero_distance_mask].copy()
cleaning_steps.append(f"Zero distance trips: {zero_distance_count:,}")
print(f"Remaining records: {len(taxi_df_clean):,}")

# 4. Remove trips with negative duration
print("\n4. REMOVING TRIPS WITH NEGATIVE DURATION")
print("-" * 50)

# Ensure trip_duration_minutes is calculated
if 'trip_duration_minutes' not in taxi_df_clean.columns:
    taxi_df_clean['trip_duration_minutes'] = (
        taxi_df_clean['tpep_dropoff_datetime'] - taxi_df_clean['tpep_pickup_datetime']
    ).dt.total_seconds() / 60

negative_duration_mask = taxi_df_clean['trip_duration_minutes'] > 0
negative_duration_count = len(taxi_df_clean) - negative_duration_mask.sum()
print(f"Records with negative duration: {negative_duration_count:,}")

taxi_df_clean = taxi_df_clean[negative_duration_mask].copy()
cleaning_steps.append(f"Negative duration trips: {negative_duration_count:,}")
print(f"Remaining records: {len(taxi_df_clean):,}")

# 5. Remove records with negative fare values
print("\n5. REMOVING RECORDS WITH NEGATIVE FARE VALUES")
print("-" * 50)

fare_cols = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount']
negative_fare_mask = pd.Series([True] * len(taxi_df_clean), index=taxi_df_clean.index)

for col in fare_cols:
    if col in taxi_df_clean.columns:
        col_mask = taxi_df_clean[col] >= 0
        negative_count = len(taxi_df_clean) - col_mask.sum()
        if negative_count > 0:
            print(f"  {col}: {negative_count:,} negative values")
        negative_fare_mask = negative_fare_mask & col_mask

total_negative_fare_count = len(taxi_df_clean) - negative_fare_mask.sum()
print(f"Total records with negative fare values: {total_negative_fare_count:,}")

taxi_df_clean = taxi_df_clean[negative_fare_mask].copy()
cleaning_steps.append(f"Negative fare values: {total_negative_fare_count:,}")
print(f"Remaining records: {len(taxi_df_clean):,}")

# 6. Remove records with missing values in critical columns
print("\n6. REMOVING RECORDS WITH MISSING VALUES")
print("-" * 50)

# Define critical columns that shouldn't have missing values
critical_cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance', 
                'fare_amount', 'total_amount', 'PULocationID', 'DOLocationID']

missing_mask = pd.Series([True] * len(taxi_df_clean), index=taxi_df_clean.index)

for col in critical_cols:
    if col in taxi_df_clean.columns:
        col_mask = taxi_df_clean[col].notna()
        missing_count = len(taxi_df_clean) - col_mask.sum()
        if missing_count > 0:
            print(f"  {col}: {missing_count:,} missing values")
        missing_mask = missing_mask & col_mask

total_missing_count = len(taxi_df_clean) - missing_mask.sum()
print(f"Total records with missing critical values: {total_missing_count:,}")

taxi_df_clean = taxi_df_clean[missing_mask].copy()
cleaning_steps.append(f"Missing critical values: {total_missing_count:,}")
print(f"Remaining records: {len(taxi_df_clean):,}")

# 7. Remove trips with 0 passengers
print("\n7. REMOVING TRIPS WITH 0 PASSENGERS")
print("-" * 50)

if 'passenger_count' in taxi_df_clean.columns:
    zero_passenger_mask = taxi_df_clean['passenger_count'] > 0
    zero_passenger_count = len(taxi_df_clean) - zero_passenger_mask.sum()
    print(f"Records with 0 passengers: {zero_passenger_count:,}")
    
    taxi_df_clean = taxi_df_clean[zero_passenger_mask].copy()
    cleaning_steps.append(f"Zero passengers: {zero_passenger_count:,}")
    print(f"Remaining records: {len(taxi_df_clean):,}")
else:
    print("passenger_count column not found, skipping this step")
    cleaning_steps.append("Zero passengers: N/A (column not found)")

# 8. Additional data quality checks
print("\n8. ADDITIONAL DATA QUALITY CHECKS")
print("-" * 50)

# Remove extremely long trips (more than 24 hours)
long_trip_mask = taxi_df_clean['trip_duration_minutes'] <= (24 * 60)
long_trip_count = len(taxi_df_clean) - long_trip_mask.sum()
if long_trip_count > 0:
    print(f"Records with trips longer than 24 hours: {long_trip_count:,}")
    taxi_df_clean = taxi_df_clean[long_trip_mask].copy()
    cleaning_steps.append(f"Trips > 24 hours: {long_trip_count:,}")
    print(f"Remaining records: {len(taxi_df_clean):,}")

# Remove trips with unrealistic distances (more than 100 miles)
distance_mask = taxi_df_clean['trip_distance'] <= 100
distance_count = len(taxi_df_clean) - distance_mask.sum()
if distance_count > 0:
    print(f"Records with trips longer than 100 miles: {distance_count:,}")
    taxi_df_clean = taxi_df_clean[distance_mask].copy()
    cleaning_steps.append(f"Trips > 100 miles: {distance_count:,}")
    print(f"Remaining records: {len(taxi_df_clean):,}")

# Summary
print("\n" + "="*80)
print("DATA CLEANING SUMMARY")
print("="*80)

print(f"Original dataset: {len(taxi_df):,} records")
print(f"Final cleaned dataset: {len(taxi_df_clean):,} records")
print(f"Total records removed: {len(taxi_df) - len(taxi_df_clean):,}")
print(f"Percentage of data retained: {len(taxi_df_clean)/len(taxi_df)*100:.2f}%")

print(f"\nCleaning steps breakdown:")
for i, step in enumerate(cleaning_steps, 1):
    print(f"  {i}. {step}")

# Reset index for clean dataset
taxi_df_clean = taxi_df_clean.reset_index(drop=True)

print(f"\nCleaned dataset info:")
print(f"Shape: {taxi_df_clean.shape}")
print(f"Date range: {taxi_df_clean['tpep_pickup_datetime'].min()} to {taxi_df_clean['tpep_pickup_datetime'].max()}")
print(f"Trip distance range: {taxi_df_clean['trip_distance'].min():.2f} to {taxi_df_clean['trip_distance'].max():.2f} miles")
print(f"Trip duration range: {taxi_df_clean['trip_duration_minutes'].min():.2f} to {taxi_df_clean['trip_duration_minutes'].max():.2f} minutes")

# Save cleaned dataset
save_cleaned = input("\nDo you want to save the cleaned dataset to a file? (y/n): ")
if save_cleaned.lower() == 'y':
    try:
        # Try saving as parquet first
        output_file = 'yellow_tripdata_2023-01_cleaned.parquet'
        
        # Drop any period columns that might cause PyArrow issues
        df_to_save = taxi_df_clean.copy()
        cols_to_drop = ['pickup_year_month', 'dropoff_year_month']
        for col in cols_to_drop:
            if col in df_to_save.columns:
                df_to_save = df_to_save.drop(columns=[col])
                print(f"Removed column '{col}' for saving")
        
        df_to_save.to_parquet(output_file, index=False)
        print(f"Cleaned dataset saved as '{output_file}'")
        
    except Exception as e:
        print(f"Error saving as parquet: {e}")
        print("Trying to save as CSV instead...")
        
        # Fallback to CSV
        output_file = 'yellow_tripdata_2023-01_cleaned.csv'
        taxi_df_clean.to_csv(output_file, index=False)
        print(f"Cleaned dataset saved as '{output_file}'")

print("\n" + "="*80)
print("DATA CLEANING COMPLETE")
print("="*80)

TAXI DATA CLEANING PROCESS
Original dataset size: 3,066,766 records

1. REMOVING RECORDS NOT IN JANUARY 2023
--------------------------------------------------
Records NOT in January 2023: 655
Remaining records: 3,066,111

2. REMOVING RECORDS WITH INVALID LOCATION IDs
--------------------------------------------------
Records with invalid location IDs: 58,218
Remaining records: 3,007,893

3. REMOVING ZERO DISTANCE TRIPS
--------------------------------------------------
Records with zero distance: 41,520
Remaining records: 2,966,373

4. REMOVING TRIPS WITH NEGATIVE DURATION
--------------------------------------------------
Records with negative duration: 56
Remaining records: 2,966,317

5. REMOVING RECORDS WITH NEGATIVE FARE VALUES
--------------------------------------------------
  fare_amount: 21,303 negative values
  extra: 10,822 negative values
  mta_tax: 21,153 negative values
  tip_amount: 30 negative values
  tolls_amount: 1,172 negative values
  total_amount: 21,404 negative

In [ ]:
# Add driver_id and customer_id columns to cleaned dataset
print("="*60)
print("ADDING DRIVER_ID AND CUSTOMER_ID COLUMNS")
print("="*60)

# Set random seed for reproducibility (optional)
np.random.seed(42)  # Remove this line if you want truly random values each time

print(f"Dataset shape before adding columns: {taxi_df_clean.shape}")

# Add driver_id column (randomized from 1 to 80,000)
taxi_df_clean['driver_id'] = np.random.randint(1, 80001, size=len(taxi_df_clean))

# Add customer_id column (randomized from 1 to 2,000,000)
taxi_df_clean['customer_id'] = np.random.randint(1, 2000001, size=len(taxi_df_clean))

print(f"Dataset shape after adding columns: {taxi_df_clean.shape}")

# Verify the new columns
print(f"\nDriver ID range: {taxi_df_clean['driver_id'].min()} to {taxi_df_clean['driver_id'].max()}")
print(f"Customer ID range: {taxi_df_clean['customer_id'].min()} to {taxi_df_clean['customer_id'].max()}")

print(f"\nUnique driver IDs: {taxi_df_clean['driver_id'].nunique():,}")
print(f"Unique customer IDs: {taxi_df_clean['customer_id'].nunique():,}")

# Show sample of the new columns
print(f"\nSample of new columns:")
sample_cols = ['driver_id', 'customer_id', 'tpep_pickup_datetime', 'PULocationID', 'DOLocationID']
print(taxi_df_clean[sample_cols].head(10))

# Show updated column list
print(f"\nUpdated columns ({len(taxi_df_clean.columns)} total):")
for i, col in enumerate(taxi_df_clean.columns, 1):
    print(f"  {i:2d}. {col}")

# Optional: Save updated dataset with new columns
save_updated = input("\nDo you want to save the dataset with new driver_id and customer_id columns? (y/n): ")
if save_updated.lower() == 'y':
    try:
        # Try saving as parquet first
        output_file = 'yellow_tripdata_2023-01_cleaned_with_ids.parquet'
        
        # Drop any period columns that might cause PyArrow issues
        df_to_save = taxi_df_clean.copy()
        cols_to_drop = ['pickup_year_month', 'dropoff_year_month']
        for col in cols_to_drop:
            if col in df_to_save.columns:
                df_to_save = df_to_save.drop(columns=[col])
                print(f"Removed column '{col}' for saving")
        
        df_to_save.to_parquet(output_file, index=False)
        print(f"Updated dataset saved as '{output_file}'")
        
    except Exception as e:
        print(f"Error saving as parquet: {e}")
        print("Trying to save as CSV instead...")
        
        # Fallback to CSV
        output_file = 'yellow_tripdata_2023-01_cleaned_with_ids.csv'
        taxi_df_clean.to_csv(output_file, index=False)
        print(f"Updated dataset saved as '{output_file}'")

print("\n" + "="*60)
print("DRIVER_ID AND CUSTOMER_ID ADDITION COMPLETE")
print("="*60)

In [ ]:
# Drop unnecessary columns to match the trips table definition
print("="*70)
print("PREPARING DATASET TO MATCH TRIPS TABLE DEFINITION")
print("="*70)

print(f"Current dataset shape: {taxi_df_clean.shape}")
print(f"Current columns: {list(taxi_df_clean.columns)}")

# Define the columns we need to keep based on the trips table definition
required_columns = [
    'driver_id',           # already added
    'customer_id',         # already added
    'VendorID',           # maps to vendorid
    'tpep_pickup_datetime',
    'tpep_dropoff_datetime', 
    'passenger_count',
    'trip_distance',
    'RatecodeID',         # maps to ratecodeid
    'store_and_fwd_flag',
    'PULocationID',       # maps to pulocationid
    'DOLocationID',       # maps to dolocationid
    'payment_type',
    'fare_amount',
    'extra',
    'mta_tax',
    'tip_amount',
    'tolls_amount',
    'improvement_surcharge',
    'total_amount',
    'congestion_surcharge',
    'airport_fee'
]

# Check which required columns exist in the dataset
existing_columns = []
missing_columns = []

for col in required_columns:
    if col in taxi_df_clean.columns:
        existing_columns.append(col)
    else:
        missing_columns.append(col)

print(f"\n--- Column Mapping Analysis ---")
print(f"Required columns: {len(required_columns)}")
print(f"Existing columns: {len(existing_columns)}")
print(f"Missing columns: {len(missing_columns)}")

if missing_columns:
    print(f"\nMissing columns:")
    for col in missing_columns:
        print(f"  - {col}")

# Check for cbd_congestion_fee (it's optional in the table definition)
if 'cbd_congestion_fee' in taxi_df_clean.columns:
    existing_columns.append('cbd_congestion_fee')
    print(f"\nOptional column 'cbd_congestion_fee' found and will be included")

# Create the filtered dataset with only required columns
taxi_df_filtered = taxi_df_clean[existing_columns].copy()

# Add trip_id as auto-incrementing primary key
taxi_df_filtered.insert(0, 'trip_id', range(1, len(taxi_df_filtered) + 1))

print(f"\n--- Dataset After Filtering ---")
print(f"New shape: {taxi_df_filtered.shape}")
print(f"Columns retained: {len(taxi_df_filtered.columns) - 1} + trip_id")  # -1 because trip_id is added

# Show the final column list
print(f"\nFinal columns ({len(taxi_df_filtered.columns)} total):")
for i, col in enumerate(taxi_df_filtered.columns, 1):
    print(f"  {i:2d}. {col}")

# Show sample of the filtered dataset
print(f"\nSample of filtered dataset:")
print(taxi_df_filtered.head())

# Show columns that were dropped
all_original_columns = set(taxi_df_clean.columns)
kept_columns = set(existing_columns)
dropped_columns = all_original_columns - kept_columns

print(f"\n--- Dropped Columns ({len(dropped_columns)} total) ---")
for col in sorted(dropped_columns):
    print(f"  - {col}")

# Data type verification for key columns
print(f"\n--- Data Type Verification ---")
print(f"trip_id: {taxi_df_filtered['trip_id'].dtype}")
print(f"driver_id: {taxi_df_filtered['driver_id'].dtype}")
print(f"customer_id: {taxi_df_filtered['customer_id'].dtype}")
if 'VendorID' in taxi_df_filtered.columns:
    print(f"VendorID: {taxi_df_filtered['VendorID'].dtype}")
if 'payment_type' in taxi_df_filtered.columns:
    print(f"payment_type: {taxi_df_filtered['payment_type'].dtype}")

# Check for any remaining data quality issues in key columns
print(f"\n--- Final Data Quality Check ---")
print(f"Total records: {len(taxi_df_filtered):,}")
print(f"trip_id range: {taxi_df_filtered['trip_id'].min()} to {taxi_df_filtered['trip_id'].max()}")
print(f"Null values by column:")
null_counts = taxi_df_filtered.isnull().sum()
for col, null_count in null_counts.items():
    if null_count > 0:
        print(f"  {col}: {null_count:,}")
    else:
        print(f"  {col}: 0")

# Optional: Save the filtered dataset
save_filtered = input("\nDo you want to save the filtered dataset ready for database import? (y/n): ")
if save_filtered.lower() == 'y':
    try:
        # Try saving as parquet first
        output_file = 'trips_table_ready.parquet'
        taxi_df_filtered.to_parquet(output_file, index=False)
        print(f"Filtered dataset saved as '{output_file}'")
        
    except Exception as e:
        print(f"Error saving as parquet: {e}")
        print("Trying to save as CSV instead...")
        
        # Fallback to CSV
        output_file = 'yellow-taxi-01-2023-cleaned.csv'
        taxi_df_filtered.to_csv(output_file, index=False)
        print(f"Filtered dataset saved as '{output_file}'")

# Store the filtered dataset for further use
trips_df = taxi_df_filtered.copy()

print("\n" + "="*70)
print("DATASET PREPARATION COMPLETE")
print("="*70)
print(f"Final dataset 'trips_df' is ready for database import")
print(f"Records: {len(trips_df):,}")
print(f"Columns: {len(trips_df.columns)}")

PREPARING DATASET TO MATCH TRIPS TABLE DEFINITION
Current dataset shape: (2832171, 28)
Current columns: ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee', 'trip_duration_minutes', 'pickup_year', 'dropoff_year', 'pickup_month', 'dropoff_month', 'pickup_year_month', 'dropoff_year_month', 'driver_id', 'customer_id']

--- Column Mapping Analysis ---
Required columns: 21
Existing columns: 21
Missing columns: 0

--- Dataset After Filtering ---
New shape: (2832171, 22)
Columns retained: 21 + trip_id

Final columns (22 total):
   1. trip_id
   2. driver_id
   3. customer_id
   4. VendorID
   5. tpep_pickup_datetime
   6. tpep_dropoff_datetime
   7. passenger_count
   8. trip_distance
   9. RatecodeID
  10. store_and_fwd_fl

In [2]:
# Remove records where PULocationID or DOLocationID is 57
print("="*70)
print("REMOVING RECORDS WITH LOCATIONID = 57")
print("="*70)

print(f"Dataset shape before removal: {trips_df.shape}")

# Check how many records have PULocationID = 57
pu_57_count = (trips_df['PULocationID'] == 57).sum()
print(f"Records with PULocationID = 57: {pu_57_count:,}")

# Check how many records have DOLocationID = 57
do_57_count = (trips_df['DOLocationID'] == 57).sum()
print(f"Records with DOLocationID = 57: {do_57_count:,}")

# Check how many records have either PULocationID = 57 OR DOLocationID = 57
either_57_count = ((trips_df['PULocationID'] == 57) | (trips_df['DOLocationID'] == 57)).sum()
print(f"Records with either PULocationID OR DOLocationID = 57: {either_57_count:,}")

# Create mask to keep records where neither PULocationID nor DOLocationID is 57
mask_keep = (trips_df['PULocationID'] != 57) & (trips_df['DOLocationID'] != 57)

# Apply the filter
trips_df_filtered = trips_df[mask_keep].copy()

# Reset index and update trip_id to maintain sequence
trips_df_filtered = trips_df_filtered.reset_index(drop=True)
trips_df_filtered['trip_id'] = range(1, len(trips_df_filtered) + 1)

print(f"\nDataset shape after removal: {trips_df_filtered.shape}")
print(f"Records removed: {len(trips_df) - len(trips_df_filtered):,}")
print(f"Percentage of data retained: {len(trips_df_filtered)/len(trips_df)*100:.2f}%")

# Verify no records with LocationID = 57 remain
remaining_pu_57 = (trips_df_filtered['PULocationID'] == 57).sum()
remaining_do_57 = (trips_df_filtered['DOLocationID'] == 57).sum()

print(f"\nVerification:")
print(f"Remaining records with PULocationID = 57: {remaining_pu_57}")
print(f"Remaining records with DOLocationID = 57: {remaining_do_57}")

# Update the main dataset
trips_df = trips_df_filtered.copy()

# Save the updated dataset
try:
    output_file = 'yellow-taxi-01-2023-cleaned-no-57.csv'
    trips_df.to_csv(output_file, index=False)
    print(f"\nUpdated dataset saved as '{output_file}'")
    
except Exception as e:
    print(f"Error saving file: {e}")

print(f"\nFinal dataset info:")
print(f"Shape: {trips_df.shape}")
print(f"trip_id range: {trips_df['trip_id'].min()} to {trips_df['trip_id'].max()}")

print("\n" + "="*70)
print("REMOVAL OF LOCATIONID = 57 COMPLETE")
print("="*70)

REMOVING RECORDS WITH LOCATIONID = 57


NameError: name 'trips_df' is not defined